## Import library

In [2]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
import warnings
import sys
sys.path.append('../utils')
from treasury_base import *

warnings.filterwarnings("ignore")

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Retrieve data

In [3]:
WINDOW_LIST = [3, 5, 10, 20]
LAG = 1

def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5
    
df_ma = ma_data_retrieval(window_list=WINDOW_LIST, lag=LAG)
df_ma.head()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,...,10 Yr_MA10,10 Yr_MA20,20 Yr_MA3,20 Yr_MA5,20 Yr_MA10,20 Yr_MA20,30 Yr_MA3,30 Yr_MA5,30 Yr_MA10,30 Yr_MA20
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-31,2.42,2.43,2.41,2.46,2.55,2.45,2.43,2.43,2.51,2.63,...,2.742,2.7180,2.906667,2.906,2.917,2.8935,3.053333,3.052,3.062,3.0375
2019-02-01,2.41,2.42,2.40,2.46,2.56,2.52,2.50,2.51,2.59,2.70,...,2.732,2.7165,2.876667,2.894,2.908,2.8935,3.030000,3.042,3.054,3.0385
2019-02-04,2.41,2.41,2.42,2.49,2.57,2.53,2.52,2.53,2.62,2.73,...,2.727,2.7235,2.870000,2.886,2.903,2.9000,3.026667,3.036,3.050,3.0440
2019-02-05,2.39,2.40,2.42,2.50,2.56,2.53,2.50,2.51,2.60,2.71,...,2.721,2.7265,2.876667,2.886,2.900,2.9045,3.026667,3.036,3.047,3.0480
2019-02-06,2.40,2.41,2.42,2.50,2.56,2.52,2.50,2.50,2.59,2.70,...,2.718,2.7270,2.896667,2.884,2.898,2.9060,3.040000,3.034,3.044,3.0500


## KAN model training

In [25]:
TEST_SIZE = 1
LENGTH = len(df_ma)
TARGETS = df_ma.columns[:12]

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': [], 'naive_mse': []}

for cnt in range(0, 20, 5):
    print()
    print('WINDOW SLIDING: ', cnt)

    df_window = df_ma[(LENGTH-cnt-250):(LENGTH-cnt)]
    # Prepare data
    X, y = df_window.drop(columns=TARGETS), df_window[TARGETS]

    # scaler = StandardScaler()
    # X = pd.DataFrame(scaler.fit_transform(X))

    n_inputs = X.shape[1]
    n_outputs = y.shape[1]

    X_train, X_test = X[:-TEST_SIZE], X[-TEST_SIZE:]
    y_train, y_test = y[:-TEST_SIZE], y[-TEST_SIZE:]

    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

    # Initialize the model
    model = KAN(width=[n_inputs, 48, 64, n_outputs], grid=4, k=2, seed=42, device=device)

    # Train the model and compute metrics
    results = model.fit(dataset, opt="Adam", lamb=0.0001, lr=0.001, steps=500, metrics=(train_mse, test_mse))
    df_naive = pd.DataFrame([y_train.iloc[-1]] * TEST_SIZE, columns=y_train.columns)
        
    # Store the metrics
    train_error = results['train_mse'][-1]
    test_error = results['test_mse'][-1]
    naive_error = mean_squared_error(df_naive, y_test, squared=False)

    fold_results['train_mse'].append(train_error)
    fold_results['test_mse'].append(test_error)
    fold_results['naive_mse'].append(naive_error)

    # Calculate average metrics across all windows
    print(f'Fold Train MSE: {train_error}')
    print(f'Fold Test MSE: {test_error}')
    print(f'Naive Test MSE: {naive_error}')

avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])
avg_naive_mse = np.mean(fold_results['naive_mse'])

print()
print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {avg_test_mse}")
print(f"Average Naive MSE: {avg_naive_mse}")

In [4]:
WINDOW_LIST = [3, 5, 10, 20]
TEST_SIZE = 5
TARGETS = df_ma.columns[:12]

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': [], 'naive_mse': []}
kan_output = []
naive_output = []
truth_value = pd.DataFrame()

for LAG in range(1, TEST_SIZE+1): # steps into the future
    df_ma = ma_data_retrieval(window_list=WINDOW_LIST, lag=LAG)

    for cnt in range(0, 40, 10): # sliding window
        print()
        print(f'WINDOW SLIDING: {cnt}, FUTURE STEPS: {LAG}')

        df_window = df_ma[(len(df_ma)-cnt-500):(len(df_ma)-cnt)]
        # Prepare data
        X, y = df_window.drop(columns=TARGETS), df_window[TARGETS]

        # scaler = StandardScaler()
        # X = pd.DataFrame(scaler.fit_transform(X))

        n_inputs = X.shape[1]
        n_outputs = y.shape[1]

        X_train, X_test = X[:-TEST_SIZE], X[-TEST_SIZE:]
        y_train, y_test = y[:-TEST_SIZE], y[-TEST_SIZE:]

        dataset = dict()
        dtype = torch.get_default_dtype()
        dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
        dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
        dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
        dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

        truth_value = pd.concat([truth_value, y_test], axis=0, ignore_index=False)

        # Initialize the model
        model = KAN(width=[n_inputs, 48, n_outputs], grid=4, k=2, seed=42, device=device)

        # Train the model and compute metrics
        results = model.fit(dataset, opt="Adam", lamb=0.0001, lr=0.003, steps=500, metrics=(train_mse, test_mse))
        
        # Naive values
        df_naive = pd.DataFrame([y_train.iloc[-LAG]] * TEST_SIZE, columns=y_train.columns)
        naive_output.append(y_train.iloc[-LAG].values)
            
        # Output values
        pred = model(dataset['test_input'])
        kan_output.append(pred.cpu().detach().numpy().flatten())

        # Store the metrics
        train_error = results['train_mse'][-1]
        test_error = results['test_mse'][-1]
        naive_error = mean_squared_error(df_naive.values.flatten(), y_test.values.flatten(), squared=False)

        fold_results['train_mse'].append(train_error)
        fold_results['test_mse'].append(test_error)
        fold_results['naive_mse'].append(naive_error)

        # Calculate average metrics across all windows
        print(f'Fold Train MSE: {train_error}')
        print(f'Fold Test MSE: {test_error}')
        print(f'Naive Test MSE: {naive_error}')

avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])
avg_naive_mse = np.mean(fold_results['naive_mse'])

print()
print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {avg_test_mse}")
print(f"Average Naive MSE: {avg_naive_mse}")


WINDOW SLIDING: 0, FUTURE STEPS: 1
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.52e-02 | test_loss: 7.76e-02 | reg: 1.30e+02 | : 100%|█| 500/500 [01:15<00:00,  6.60


saving model version 0.1
Fold Train MSE: 0.06739296713116193
Fold Test MSE: 0.07756239893047981
Naive Test MSE: 0.07320063751999253

WINDOW SLIDING: 10, FUTURE STEPS: 1
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.26e-02 | test_loss: 3.95e-02 | reg: 1.31e+02 | : 100%|█| 500/500 [01:07<00:00,  7.45


saving model version 0.1
Fold Train MSE: 0.0624594365577303
Fold Test MSE: 0.039512812555534606
Naive Test MSE: 0.029608557321603283

WINDOW SLIDING: 20, FUTURE STEPS: 1
checkpoint directory created: ./model
saving model version 0.0


...

saving model version 0.1
Fold Train MSE: 0.065513897038592
Fold Test MSE: 0.053132953052345534
Naive Test MSE: 0.08250252521387859

WINDOW SLIDING: 20, FUTURE STEPS: 5
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.52e-02 | test_loss: 1.07e-01 | reg: 1.43e+02 | : 100%|█| 500/500 [01:04<00:00,  7.70


saving model version 0.1
Fold Train MSE: 0.06647521643372895
Fold Test MSE: 0.10727819613792891
Naive Test MSE: 0.11414172476939945

WINDOW SLIDING: 30, FUTURE STEPS: 5
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.42e-02 | test_loss: 1.53e-01 | reg: 1.44e+02 | : 100%|█| 500/500 [01:03<00:00,  7.86


saving model version 0.1
Fold Train MSE: 0.063955448408842
Fold Test MSE: 0.15338353205867428
Naive Test MSE: 0.07786740867226376

Sliding Window Cross-Validation Results
Average Train MSE: 0.06748961815658433
Average Test MSE: 0.08673105527001192
Average Naive MSE: 0.08603590116067626


In [5]:
import pickle

kan_df = pd.DataFrame(kan_output)
naive_df = pd.DataFrame(naive_output)

dataframes = {
    "naive_df": naive_df,
    "kan_df": kan_df,
    "truth_value": truth_value
}

# Specify the file name
filename = "dataframes.pkl"

# Pickle the DataFrames into a file
with open(filename, "wb") as file:
    pickle.dump(dataframes, file)

print(f"DataFrames have been saved to {filename}")

DataFrames have been saved to dataframes.pkl


In [68]:
with open("dfs_step_1.pkl", "rb") as file:
    loaded_dataframes = pickle.load(file)

# Extract individual DataFrames
naive_df = loaded_dataframes["naive_df"]
kan_df = loaded_dataframes["kan_df"]
truth_value = loaded_dataframes["truth_value"]

# Use the DataFrames
truth_value

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,
2024-12-06,4.57,4.50,4.42,4.34,4.19,4.10,4.05,4.03,4.09,4.15,4.42,4.34
2024-11-21,4.72,4.67,4.63,4.45,4.39,4.34,4.30,4.30,4.36,4.43,4.68,4.61
2024-11-06,4.68,4.71,4.64,4.41,4.31,4.27,4.20,4.27,4.37,4.42,4.71,4.60
2024-10-23,4.88,4.80,4.73,4.48,4.27,4.07,4.03,4.05,4.14,4.24,4.58,4.51


In [59]:
truth_value

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,
2024-12-06,4.57,4.50,4.42,4.34,4.19,4.10,4.05,4.03,4.09,4.15,4.42,4.34
2024-11-21,4.72,4.67,4.63,4.45,4.39,4.34,4.30,4.30,4.36,4.43,4.68,4.61
2024-11-06,4.68,4.71,4.64,4.41,4.31,4.27,4.20,4.27,4.37,4.42,4.71,4.60
2024-10-23,4.88,4.80,4.73,4.48,4.27,4.07,4.03,4.05,4.14,4.24,4.58,4.51


In [60]:
pd.DataFrame(kan_output)

""


In [43]:
pd.DataFrame(naive_output)

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.59,4.53,4.46,4.38,4.23,4.15,4.10,4.07,4.12,4.17,4.43,4.33
1,4.68,4.63,4.62,4.44,4.37,4.31,4.26,4.28,4.34,4.41,4.66,4.59
2,4.72,4.72,4.64,4.39,4.27,4.19,4.11,4.16,4.22,4.26,4.55,4.44
3,4.89,4.81,4.72,4.47,4.24,4.03,3.98,4.00,4.10,4.20,4.55,4.49


In [ ]:
WINDOW_LIST = [1]
TEST_SIZE = 20
TARGETS = df_ma.columns[:12]

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': [], 'naive_mse': []}

for LAG in range(1, 2): # steps into the future
    df_ma = ma_data_retrieval(window_list=WINDOW_LIST, lag=LAG)

    for cnt in range(0, 20, 20): # sliding window
        print()
        print(f'WINDOW SLIDING: {cnt}, LAG: {LAG}')

        df_window = df_ma[(len(df_ma)-cnt-250):(len(df_ma)-cnt)]

X, y = df_window.drop(columns=TARGETS), df_window[TARGETS]

X_train, X_test = X[:-TEST_SIZE], X[-TEST_SIZE:]
y_train, y_test = y[:-TEST_SIZE], y[-TEST_SIZE:]

df_ma

In [9]:
pred = model(dataset['test_input'])
pred.cpu().detach().numpy().flatten()

array([ 3.23854795, -1.30375493,  2.63067184,  2.239742  ,  2.18245819,
        1.15600575,  0.21529186,  3.55610411,  4.71038988,  4.07025871,
        3.55403183,  2.4741612 ])

In [12]:
y_test

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,
2024-09-11,5.21,5.19,5.10,4.72,4.12,3.62,3.45,3.45,3.54,3.65,4.03,3.96
2024-09-12,5.18,5.22,5.06,4.68,4.09,3.64,3.47,3.47,3.57,3.68,4.07,4.00
2024-09-13,5.15,5.17,4.97,4.60,4.00,3.57,3.42,3.43,3.53,3.66,4.05,3.98
2024-09-16,5.11,5.10,4.96,4.55,3.96,3.56,3.42,3.41,3.51,3.63,4.01,3.94
2024-09-17,5.05,5.05,4.95,4.55,3.99,3.59,3.45,3.44,3.53,3.65,4.02,3.96
2024-09-18,4.91,4.91,4.84,4.50,3.95,3.61,3.49,3.47,3.58,3.70,4.08,4.03
2024-09-19,4.89,4.91,4.80,4.46,3.93,3.59,3.47,3.49,3.60,3.73,4.11,4.06
2024-09-20,4.87,4.88,4.75,4.43,3.92,3.55,3.46,3.48,3.59,3.73,4.10,4.07
2024-09-23,4.85,4.84,4.72,4.40,3.91,3.57,3.47,3.51,3.62,3.75,4.12,4.09


In [90]:
mean_squared_error(df_naive.values.flatten(), y_test.values.flatten(), squared=False)

np.float64(0.28945350806879727)

In [92]:
x1 = df_naive.values.flatten()
x2 = y_test.values.flatten()
print(x1)
print(x2)

[4.96 4.85 4.75 4.44 4.21 3.98 3.86 3.86 3.94 4.04 4.38 4.32]
[4.68 4.71 4.64 4.41 4.31 4.27 4.2  4.27 4.37 4.42 4.71 4.6 ]


## Optuna training

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

In [ ]:
import optuna
import torch

def train_mse(model, dataset):
    predictions = model(dataset['train_input'])  # Model predictions
    loss = torch.nn.functional.mse_loss(predictions, dataset['train_label'])
    return loss

def test_mse(model, dataset):
    predictions = model(dataset['test_input'])  # Model predictions
    loss = torch.nn.functional.mse_loss(predictions, dataset['test_label'])
    return loss

# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    n_layers = trial.suggest_int('n_layers', 1, 2)  # Number of layers in the network
    layer_sizes = [trial.suggest_int(f'n_units_l{i}', 16, 64, step=16) for i in range(n_layers)]
    grid = trial.suggest_int('grid', 2, 4)          # Example parameter for KAN
    lamb = trial.suggest_float('lamb', 1e-4, 1e-2, log=True)  # Regularization rate
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)       # Learning rate
    steps = trial.suggest_int('steps', 500, 2000, step=500)   # Training steps

    # Model architecture
    width = [n_inputs] + layer_sizes + [n_outputs]

    # Initialize dataset
    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

    # Initialize the model
    model = KAN(width=width, grid=grid, k=2, seed=42, device=device)

    # Train the model
    results = model.fit(
        dataset, 
        opt="Adam", 
        lamb=lamb, 
        lr=lr, 
        steps=steps, 
        metrics=(lambda: train_mse(model, dataset), lambda: test_mse(model, dataset))
    )

    # Retrieve the metric (e.g., test MSE) from the results
    test_mse_value = results['test_loss'][-1]
    return test_mse_value  # Minimize test MSE

# Create an Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best parameters and results
print("Best parameters:", study.best_params)
print("Best test MSE:", study.best_value)
